In [ ]:
import os
import subprocess

# נתיב ל-sox.exe
sox_path = r"C:\Program Files (x86)\sox-14-4-2\sox.exe"

# תיקיות מקור ויעד
dir_sph = r"TEDLIUM_release-3\data\sph"
dir_wav = r"TEDLIUM_release-3\data\wav"

# יצירת תיקיית WAV אם לא קיימת
os.makedirs(dir_wav, exist_ok=True)

# כל קובצי SPH בתיקייה
sph_files = sorted([f for f in os.listdir(dir_sph) if f.endswith('.sph')])

# המרה
for sph_file in sph_files:
    sph_path = os.path.join(dir_sph, sph_file)
    wav_file = os.path.splitext(sph_file)[0] + '.wav'
    wav_path = os.path.join(dir_wav, wav_file)

    if os.path.exists(wav_path):
        print(f"⏭️ מדלג על {wav_file} – כבר קיים.")
        continue

    try:
        subprocess.run([sox_path, sph_path, wav_path], check=True)
        print(f"✅ {sph_file} → {wav_file}")
    except subprocess.CalledProcessError as e:
        print(f"❌ שגיאה בהמרת {sph_file}: {e}")C:\Users\Yonatan\tfgpu\Scripts\activate



⏭️ מדלג על 911Mothers_2010W.wav – כבר קיים.
⏭️ מדלג על AJJacobs_2007P.wav – כבר קיים.
⏭️ מדלג על AJJacobs_2011P.wav – כבר קיים.
⏭️ מדלג על AJJacobs_2014A.wav – כבר קיים.
⏭️ מדלג על AalaElKhani_2016X.wav – כבר קיים.
⏭️ מדלג על AaronHuey_2010X.wav – כבר קיים.
⏭️ מדלג על AaronKoblin_2011.wav – כבר קיים.
⏭️ מדלג על AaronOConnell_2011.wav – כבר קיים.
⏭️ מדלג על AbeDavis_2015.wav – כבר קיים.
⏭️ מדלג על AbhaDawesar_2013G.wav – כבר קיים.
⏭️ מדלג על AbigailMarsh_2016T.wav – כבר קיים.
⏭️ מדלג על AbigailWashburn_2012U.wav – כבר קיים.
⏭️ מדלג על AbrahamVerghese_2011G.wav – כבר קיים.
⏭️ מדלג על AchenyoIdachaba_2015W.wav – כבר קיים.
⏭️ מדלג על AdamDavidson_2012S.wav – כבר קיים.
⏭️ מדלג על AdamDriver_2015P.wav – כבר קיים.
⏭️ מדלג על AdamFoss_2016.wav – כבר קיים.
⏭️ מדלג על AdamGalinsky_2016X.wav – כבר קיים.
⏭️ מדלג על AdamGarone_2011X.wav – כבר קיים.
⏭️ מדלג על AdamGrant_2016.wav – כבר קיים.
⏭️ מדלג על AdamGrant_2016S.wav – כבר קיים.
⏭️ מדלג על AdamOckelford_2013X.wav – כבר קיים.
⏭️ מדלג על AdamOstro

In [ ]:
# ╔════════════════════════════════════════════════════════════════╗
# ║  6 demo clips – two for each noise type (play original + noisy)║
# ╚════════════════════════════════════════════════════════════════╝
import os, random, uuid
from pydub import AudioSegment
from IPython.display import Audio, display

# ─── נתיבים ──────────────────────────────────────────────────────
WAV_DIR = r"F:/DEPP/data2/TEDLIUM_release-3/data/wav"  # קבצי WAV מקוריים
TMP_DIR = "_preview_clips"
os.makedirs(TMP_DIR, exist_ok=True)

# ─── פרמטרים ─────────────────────────────────────────────────────
NOISE_TYPES        = ["signal_loss", "volume_drop", "compression_artifact"]
MIN_NOISE_DURATION = 3.0   # שניות
MAX_NOISE_DURATION = 4.0

# ─── פונקציות רעש ────────────────────────────────────────────────
def apply_signal_loss(aud, s, e):
    return aud[:int(s*1000)] + AudioSegment.silent(int((e-s)*1000)) + aud[int(e*1000):]

def apply_volume_drop(aud, s, e):
    seg = aud[int(s*1000): int(e*1000)]
    seg = seg.fade_out(min(300, len(seg)//2)).fade_in(min(300, len(seg)//2)) - 23  # ‼️ -23 dB
    return aud[:int(s*1000)] + seg + aud[int(e*1000):]

def apply_compression_artifact(aud, s, e):
    seg = aud[int(s*1000): int(e*1000)]
    seg = (seg.set_sample_width(1)
              .set_frame_rate(5000)
              .low_pass_filter(2000) - 20)
    return aud[:int(s*1000)] + seg + aud[int(e*1000):]

OPERATORS = {
    "signal_loss":          apply_signal_loss,
    "volume_drop":          apply_volume_drop,
    "compression_artifact": apply_compression_artifact,
}

random.seed(42)

# ─── יצירת הדוגמאות והשמעה ──────────────────────────────────────
wav_files = [f for f in os.listdir(WAV_DIR) if f.endswith(".wav")]
if not wav_files:
    raise RuntimeError(f"No WAV files found in {WAV_DIR}")

for ntype in NOISE_TYPES:
    print(f"\n==============================")
    print(f"🎯 Noise Type: {ntype}")
    print(f"==============================")
    
    for _ in range(2):  # שתי דגימות מכל סוג
        wav_file = random.choice(wav_files)
        path     = os.path.join(WAV_DIR, wav_file)
        audio    = AudioSegment.from_file(path)
        dur      = len(audio) / 1000.0
        
        ndur  = random.uniform(MIN_NOISE_DURATION, MAX_NOISE_DURATION)
        start = random.uniform(0, max(0.1, dur - ndur))
        end   = start + ndur

        noisy_audio = OPERATORS[ntype](audio, start, end)

        orig_seg  = audio[int(start*1000): int(end*1000)]
        noisy_seg = noisy_audio[int(start*1000): int(end*1000)]

        uid        = uuid.uuid4().hex[:8]
        orig_path  = os.path.join(TMP_DIR, f"orig_{ntype}_{uid}.wav")
        noisy_path = os.path.join(TMP_DIR, f"noisy_{ntype}_{uid}.wav")
        orig_seg.export(orig_path,  format="wav")
        noisy_seg.export(noisy_path, format="wav")

        print(f"\n📁 File: {wav_file}")
        print(f"🕒 Time: {start:.2f}s – {end:.2f}s  (≈{ndur:.1f}s)")
        print("🔊 Original segment:")
        display(Audio(orig_path))
        print("🔊 Noisy segment:")
        display(Audio(noisy_path))

print("\n💡 כשהחלטת אם זה טוב, אפשר למחוק את תיקיית", TMP_DIR)



🎯 Noise Type: signal_loss

📁 File: DanielLevitin_2015G.wav
🕒 Time: 203.89s – 206.91s  (≈3.0s)
🔊 Original segment:


🔊 Noisy segment:



📁 File: JasonClay_2010G.wav
🕒 Time: 119.24s – 122.38s  (≈3.1s)
🔊 Original segment:


🔊 Noisy segment:



🎯 Noise Type: volume_drop

📁 File: ChiekoAsakawa_2015S.wav
🕒 Time: 18.11s – 21.70s  (≈3.6s)
🔊 Original segment:


...


🎯 Noise Type: compression_artifact

📁 File: AndersYnnerman_2010X.wav
🕒 Time: 708.66s – 712.22s  (≈3.6s)
🔊 Original segment:


🔊 Noisy segment:



📁 File: RLukeDuBois_2016.wav
🕒 Time: 450.29s – 453.51s  (≈3.2s)
🔊 Original segment:


🔊 Noisy segment:



💡 כשהחלטת אם זה טוב, אפשר למחוק את תיקיית _preview_clips


In [3]:
"""
Create FULL FILES with multiple noises - WHOLE FILE APPROACH
-----------------------------------------------------------
• כל קובץ: 15 רעשים לפי התפלגות קיימת
• כל רעש: 3-4 שניות
• לייבל מדויק: רשומה נפרדת לכל רעש עם זמנים מדויקים
• יצוא:
      dataset_v3/data/*.wav (קבצים שלמים עם רעשים)
      dataset_v3/labels.csv (פורמט משופר)
"""
import os, random, csv, uuid
from pathlib import Path
from typing import Dict, List
from pydub import AudioSegment

# ───────── CONFIG ────────────────────────────────────────────────
WAV_DIR        = Path(r"F:/DEPP/data2/TEDLIUM_release-3/data/wav")
OUTPUT_DIR     = Path("dataset")    # שם התיקייה dataset
MIN_NOISES_PER_FILE = 10            # מינימום רעשים לקובץ
MAX_NOISES_PER_FILE = 15            # מקסימום רעשים לקובץ

# כל רעש 3-4 שניות (כמו בתא השני!)
NOISE_MIN, NOISE_MAX = 3.0, 4.0

# התפלגות זהה לתא השני
MIX: Dict[str, float] = {           
    "compression_artifact": 0.50,  
    "volume_drop":          0.35,  
    "signal_loss":          0.15,  
}

# הפרשים בין רעשים
MIN_NOISE_SPACING = 15.0  # 15 שניות מינימום בין רעשים

DATA_DIR  = OUTPUT_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
LABEL_CSV = OUTPUT_DIR / "labels.csv"

# ───────── NOISE OPERATIONS (זהה לתא השני!) ────────────────────
def apply_signal_loss(audio, start_s, end_s):
    before = audio[:int(start_s*1000)]
    target = AudioSegment.silent(duration=int((end_s-start_s)*1000))
    after = audio[int(end_s*1000):]
    return before + target + after

def apply_volume_drop(audio, start_s, end_s):
    before = audio[:int(start_s*1000)]
    target = audio[int(start_s*1000):int(end_s*1000)]
    fade_duration = min(300, len(target)//2)
    target = target.fade_out(fade_duration).fade_in(fade_duration) - 23
    after = audio[int(end_s*1000):]
    return before + target + after

def apply_compression_artifact(audio, start_s, end_s):
    before = audio[:int(start_s*1000)]
    target = audio[int(start_s*1000):int(end_s*1000)]
    target = (target.set_sample_width(1)
                   .set_frame_rate(5000)
                   .low_pass_filter(2000) - 20)
    after = audio[int(end_s*1000):]
    return before + target + after

NOISE_FUNCTIONS = {
    "signal_loss":          apply_signal_loss,
    "volume_drop":          apply_volume_drop,
    "compression_artifact": apply_compression_artifact,
}

# ───────── HELPER FUNCTIONS ─────────────────────────────────────
def time_to_mmss(seconds):
    """המרת שניות לפורמט MM:SS"""
    mm = int(seconds // 60)
    ss = int(seconds % 60)
    return f"{mm:02d}:{ss:02d}"

def generate_noise_events(audio_duration: float, num_noises: int) -> List[Dict]:
    """יוצר רשימת אירועי רעש לקובץ שלם"""
    noise_types = list(MIX.keys())
    noise_weights = list(MIX.values())
    
    noise_events = []
    used_starts = []
    
    for _ in range(num_noises):
        # ניסיון למצוא מיקום מתאים
        for attempt in range(100):  # מקסימום 100 ניסיונות
            noise_duration = random.uniform(NOISE_MIN, NOISE_MAX)
            
            if noise_duration >= audio_duration:
                continue
                
            start_time = random.uniform(0, audio_duration - noise_duration)
            
            # בדיקה שיש מרווח מספיק מרעשים קיימים
            if all(abs(start_time - existing) >= MIN_NOISE_SPACING for existing in used_starts):
                end_time = start_time + noise_duration
                noise_type = random.choices(noise_types, weights=noise_weights, k=1)[0]
                
                noise_events.append({
                    "start_time": round(start_time, 3),
                    "end_time": round(end_time, 3),
                    "duration": round(noise_duration, 3),
                    "noise_type": noise_type
                })
                
                used_starts.append(start_time)
                break
    
    # מיון לפי זמן התחלה
    noise_events.sort(key=lambda x: x["start_time"])
    return noise_events

# ───────── MAIN PROCESSING ──────────────────────────────────────
sources = [p for p in WAV_DIR.iterdir() if p.suffix == ".wav"]
if not sources:
    raise RuntimeError(f"No WAV files found in {WAV_DIR}")

labels = []
random.seed(42)  # למען שחזור

print(f"🎯 Creating full files with 10-15 noises each...")
print(f"▶ Processing {len(sources)} WAV files")
print(f"📊 Noise distribution: {MIX}")

for src_idx, src in enumerate(sources):
    if src_idx % 50 == 0:
        print(f"   📁 Processing file {src_idx+1}/{len(sources)}: {src.name}")
    
    try:
        # טעינת הקובץ המקורי
        audio = AudioSegment.from_file(src)
        audio_duration = len(audio) / 1000.0  # seconds
        
        # קביעת מספר רעשים לקובץ הזה
        num_noises = random.randint(MIN_NOISES_PER_FILE, MAX_NOISES_PER_FILE)
        
        # יצירת אירועי רעש
        noise_events = generate_noise_events(audio_duration, num_noises)
        
        if len(noise_events) < 8:  # לפחות 8 רעשים
            print(f"   ⚠️ Skipping {src.name} - only {len(noise_events)} noises generated")
            continue
        
        # יישום הרעשים על הקובץ
        noisy_audio = audio
        for event in noise_events:
            start_s = event["start_time"]
            end_s = event["end_time"]
            noise_type = event["noise_type"]
            
            noisy_audio = NOISE_FUNCTIONS[noise_type](noisy_audio, start_s, end_s)
        
        # שמירת הקובץ עם הרעשים
        output_filename = f"{src.stem}_with_noises.wav"
        output_path = DATA_DIR / output_filename
        noisy_audio.export(output_path, format="wav")
        
        # הוספת לייבלים - רשומה נפרדת לכל רעש
        for event in noise_events:
            labels.append({
                "filename": output_filename,
                "noise_type": event["noise_type"],
                "start_time": event["start_time"],
                "end_time": event["end_time"],
                "duration": event["duration"],
                "start_time_mmss": time_to_mmss(event["start_time"]),
                "end_time_mmss": time_to_mmss(event["end_time"])
            })
            
    except Exception as e:
        print(f"   ❌ Error processing {src.name}: {e}")
        continue

# ───────── SAVE LABELS CSV ───────────────────────────────────────
fieldnames = ["filename", "noise_type", "start_time", "end_time", "duration", "start_time_mmss", "end_time_mmss"]

with open(LABEL_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(labels)

print(f"\n✅ Done! Full files with noises created:")
print(f"   📁 Files dir: {DATA_DIR.resolve()}")
print(f"   📋 Labels: {LABEL_CSV.resolve()}")
print(f"   📊 Total files: {len(set(label['filename'] for label in labels))}")
print(f"   📊 Total noise instances: {len(labels)}")
print(f"   📊 Average noises per file: {len(labels) / len(set(label['filename'] for label in labels)):.1f}")


🎯 Creating full files with 10-15 noises each...
▶ Processing 2200 WAV files
📊 Noise distribution: {'compression_artifact': 0.5, 'volume_drop': 0.35, 'signal_loss': 0.15}
   📁 Processing file 1/2200: 911Mothers_2010W.wav
   📁 Processing file 51/2200: AlexaMeade_2013G.wav
   📁 Processing file 51/2200: AlexaMeade_2013G.wav
   📁 Processing file 101/2200: AnandaShankar_2009I.wav
   📁 Processing file 101/2200: AnandaShankar_2009I.wav
   📁 Processing file 151/2200: AnnMorgan_2015G.wav
   📁 Processing file 151/2200: AnnMorgan_2015G.wav
   📁 Processing file 201/2200: BartKnols_2012X.wav
   📁 Processing file 201/2200: BartKnols_2012X.wav
   📁 Processing file 251/2200: BillyGraham_1998.wav
   📁 Processing file 251/2200: BillyGraham_1998.wav
   📁 Processing file 301/2200: CameronHerold_2009X.wav
   📁 Processing file 301/2200: CameronHerold_2009X.wav
   📁 Processing file 351/2200: CharlieRose_2014.wav
   📁 Processing file 351/2200: CharlieRose_2014.wav
   📁 Processing file 401/2200: ClayShirky_2010

In [7]:
import pandas as pd
from pathlib import Path

LABELS_PATH = Path("dataset/labels.csv")   # עדכן אם הקובץ במקום אחר

df = pd.read_csv(LABELS_PATH)

# ספירת המופעים של כל סוג רעש
counts = df["noise_type"].value_counts()

print("📊 Number of noise segments per type:")
print(counts)


📊 Number of noise segments per type:
noise_type
compression_artifact    13837
volume_drop              9555
signal_loss              4038
Name: count, dtype: int64


In [8]:
import os
import librosa
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm

# ───────── PATHS ─────────────────────────────────────────────────
AUDIO_DIR   = "dataset/data"
LABELS_PATH = "dataset/labels.csv"
OUTPUT_DIR  = "preprocessed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ───────── AUDIO PARAMS ──────────────────────────────────────────
SR         = 16000
N_MELS     = 64
HOP_LENGTH = 160
N_FFT      = 400

# ───────── NOISE MAPPING ─────────────────────────────────────────
noise_to_idx = {
    'signal_loss':          0,
    'volume_drop':          1,
    'compression_artifact': 2
}

print("🔍 Loading labels and file list...")
labels_df = pd.read_csv(LABELS_PATH)

file_list = sorted([f for f in os.listdir(AUDIO_DIR) if f.endswith(".wav")])
total_files = len(file_list)

# Check how many already exist
existing = sum(1 for f in file_list 
               if os.path.exists(os.path.join(OUTPUT_DIR, f"{os.path.splitext(f)[0]}_X.npy")))

print(f"📊 Found {total_files} audio files")
print(f"📋 {existing} files already processed")
print(f"🎯 {total_files - existing} files to process")

if total_files == existing:
    print("✅ All files already processed!")
else:
    print("🚀 Starting sequential processing...")
    
    successful = 0
    skipped = 0
    errors = []
    
    # Process files one by one with progress bar
    for filename in tqdm(file_list, desc="Processing files"):
        name = os.path.splitext(filename)[0]
        x_path = os.path.join(OUTPUT_DIR, f"{name}_X.npy")
        y_path = os.path.join(OUTPUT_DIR, f"{name}_y.npy")

        # Skip if already exists
        if os.path.exists(x_path) and os.path.exists(y_path):
            skipped += 1
            continue

        path = os.path.join(AUDIO_DIR, filename)
        
        try:
            # Load audio with optimized settings
            y_audio, _ = librosa.load(path, sr=SR, mono=True, dtype=np.float32)
            
            if len(y_audio) < N_FFT:
                errors.append(f"{filename}: too_short")
                continue

            # Optimized spectrogram computation
            mel = librosa.feature.melspectrogram(
                y=y_audio, sr=SR, n_fft=N_FFT,
                hop_length=HOP_LENGTH, n_mels=N_MELS,
                power=2.0  # Faster than default
            )
            
            # Log conversion and normalization
            log_mel = librosa.power_to_db(mel, ref=np.max)
            log_mel = (log_mel + 80) / 80  # Fixed normalization [-80, 0] -> [0, 1]
            log_mel = np.clip(log_mel, 0, 1)
            
            T = log_mel.shape[1]
            
            # Create label matrix
            label_matrix = np.zeros((T, 3), dtype=np.float32)
            file_labels = labels_df[labels_df['filename'] == filename]
            
            for _, row in file_labels.iterrows():
                start_sec, end_sec = float(row['start_time']), float(row['end_time'])
                ntype = row['noise_type']

                if ntype not in noise_to_idx:
                    continue

                start_idx = librosa.time_to_frames(start_sec, sr=SR, hop_length=HOP_LENGTH)
                end_idx = librosa.time_to_frames(end_sec, sr=SR, hop_length=HOP_LENGTH)
                start_idx = max(0, min(start_idx, T-1))
                end_idx = max(start_idx+1, min(end_idx, T))

                if start_idx < end_idx:
                    label_matrix[start_idx:end_idx, noise_to_idx[ntype]] = 1.0

            # Save with compression
            np.save(x_path, log_mel.astype(np.float32))
            np.save(y_path, label_matrix)
            
            successful += 1
            
            # Memory cleanup every 50 files
            if successful % 50 == 0:
                del y_audio, mel, log_mel, label_matrix
                gc.collect()
                
        except Exception as e:
            errors.append(f"{filename}: {str(e)}")
            continue

    # Final summary
    print(f"\n🎉 Processing Complete!")
    print(f"   ✅ Successfully processed: {successful}")
    print(f"   ⏭️ Skipped (already done): {skipped}")
    print(f"   ❌ Errors: {len(errors)}")
    print(f"   📁 Output directory: {os.path.abspath(OUTPUT_DIR)}")
    
    if errors:
        print(f"\n⚠️ Error details:")
        for error in errors[:5]:  # Show first 5 errors
            print(f"   {error}")
        if len(errors) > 5:
            print(f"   ... and {len(errors)-5} more errors")

# Force cleanup
gc.collect()

🔍 Loading labels and file list...
📊 Found 2200 audio files
📋 2200 files already processed
🎯 0 files to process
✅ All files already processed!
📊 Found 2200 audio files
📋 2200 files already processed
🎯 0 files to process
✅ All files already processed!


1282

In [2]:
import os
import numpy as np

# Directory containing the processed label files (*.npy)
PREPROCESSED_DIR = "preprocessed"

# Initialize counters
total_frames = 0
frames_with_noise = 0

print("🔍 Scanning label files in:", PREPROCESSED_DIR)

for filename in sorted(os.listdir(PREPROCESSED_DIR)):
    if filename.endswith("_y.npy"):
        y_path = os.path.join(PREPROCESSED_DIR, filename)
        y = np.load(y_path)  # Shape expected: (T, 3)

        # Total frames in this file
        T = y.shape[0]
        total_frames += T

        # Count how many frames have at least one type of noise (any of the 3 is 1)
        noisy = np.any(y > 0.5, axis=1)  # Boolean array: True if any label in frame is 1
        num_noisy_frames = np.sum(noisy)

        frames_with_noise += num_noisy_frames

# Compute final statistics
clean_frames = total_frames - frames_with_noise
noise_percent = (frames_with_noise / total_frames) * 100
clean_percent = (clean_frames / total_frames) * 100
diff = abs(clean_frames - frames_with_noise)

# Print results nicely
print("\n📊 Label Distribution Summary (Multi-label, Frame-based):")
print(f"🔢 Total frames: {total_frames:,}")
print(f"✅ Frames with noise (any type): {frames_with_noise:,}  ({noise_percent:.2f}%)")
print(f"🔸 Clean frames (no noise):       {clean_frames:,}  ({clean_percent:.2f}%)")
print(f"📉 Difference between clean and noisy frames: {diff:,} frames")


🔍 Scanning label files in: preprocessed

📊 Label Distribution Summary (Multi-label, Frame-based):
🔢 Total frames: 190,833,660
✅ Frames with noise (any type): 9,610,992  (5.04%)
🔸 Clean frames (no noise):       181,222,668  (94.96%)
📉 Difference between clean and noisy frames: 171,611,676 frames

📊 Label Distribution Summary (Multi-label, Frame-based):
🔢 Total frames: 190,833,660
✅ Frames with noise (any type): 9,610,992  (5.04%)
🔸 Clean frames (no noise):       181,222,668  (94.96%)
📉 Difference between clean and noisy frames: 171,611,676 frames


In [1]:
import os, random, math
import numpy as np
import tensorflow as tf

# ═══════════════════════════════════════════════════════════════════
# 🎯 IMPROVED AUDIO NOISE DETECTION MODEL - DATA PREPARATION FOR CPU
# ═══════════════════════════════════════════════════════════════════

DATA_DIR   = "preprocessed"
MODEL_DIR  = "models"; os.makedirs(MODEL_DIR, exist_ok=True)

# CPU-optimized settings
MAX_T      = 800      # Good temporal context
BATCH_SIZE = 4        # Smaller batch for CPU (was 6)

# ───────── DATASET SPLIT - USING ALL 2200 FILES ─────────────────────
X_files = sorted(f for f in os.listdir(DATA_DIR) if f.endswith("_X.npy"))
y_files = sorted(f for f in os.listdir(DATA_DIR) if f.endswith("_y.npy"))

print(f"📊 Total available files: {len(X_files)}")

# Use ALL available data with 15% validation split
total_files = len(X_files)
val_split = int(0.85 * total_files)  # 85% train, 15% validation

idx = np.arange(total_files)
np.random.seed(42)  # For reproducibility
np.random.shuffle(idx)

train_idx = idx[:val_split]     # ~1870 files for training
val_idx = idx[val_split:]       # ~330 files for validation

print(f"🎯 Training files: {len(train_idx)} ({len(train_idx)/total_files*100:.1f}%)")
print(f"📋 Validation files: {len(val_idx)} ({len(val_idx)/total_files*100:.1f}%)")

# ───────── OPTIMIZED DATA LOADING ────────────────────────────────────
def load_xy(i):
    """Optimized data loading with error handling"""
    try:
        X = np.load(os.path.join(DATA_DIR, X_files[i])).astype(np.float32)
        y = np.load(os.path.join(DATA_DIR, y_files[i])).astype(np.float32)
        
        # Ensure correct shape (T, 3)
        if y.shape[0] == 3 and y.shape[1] != 3: 
            y = y.T
            
        # Adaptive truncation based on file length
        actual_T = min(X.shape[1], y.shape[0], MAX_T)
        X = X[:, :actual_T]
        y = y[:actual_T]
        
        return X[..., None], y
    except Exception as e:
        print(f"Error loading file {i}: {e}")
        # Return empty data that will be filtered out
        return np.zeros((64, 1, 1), dtype=np.float32), np.zeros((1, 3), dtype=np.float32)

# ───────── IMPROVED AUGMENTATION TECHNIQUES FOR CPU ──────────────────────────
def additive_noise(X, lvl_range=(0.005, 0.02)):
    """CPU-friendly additive noise with variable intensity"""
    lvl = random.uniform(*lvl_range)
    return np.clip(X + np.random.normal(0, lvl, X.shape), 0, 1)

def random_intensity(X, intensity_range=(0.9, 1.1)):
    """Conservative intensity scaling for CPU"""
    scale = random.uniform(*intensity_range)
    return np.clip(X * scale, 0, 1)

def time_shift(X, y, max_shift=15):
    """CPU-optimized time shifting"""
    s = random.randint(-max_shift, max_shift)
    if s != 0: 
        X = np.roll(X, s, axis=1)
        y = np.roll(y, s, axis=0)
    return X, y

def frequency_mask(X, mask_prob=0.3):
    """CPU-friendly frequency masking"""
    if random.random() < mask_prob:
        mask_height = random.randint(2, X.shape[0] // 5)
        mask_start = random.randint(0, X.shape[0] - mask_height)
        X[mask_start:mask_start + mask_height, :, 0] = 0
    return X

def time_mask(X, mask_prob=0.25):
    """CPU-friendly time masking"""
    if random.random() < mask_prob:
        mask_width = random.randint(3, min(25, X.shape[1] // 8))
        mask_start = random.randint(0, max(1, X.shape[1] - mask_width))
        X[:, mask_start:mask_start + mask_width, 0] = 0
    return X

def compression_augment(X, strength_range=(0.5, 0.75)):
    """Optimized compression artifact simulation"""
    cutoff = random.uniform(*strength_range)
    freq_cutoff = int(X.shape[0] * cutoff)
    X[freq_cutoff:, :, 0] *= random.uniform(0.2, 0.4)
    # Light compression noise
    X += np.random.normal(0, 0.005, X.shape)
    return np.clip(X, 0, 1)

def volume_drop_augment(X, db_range=(10, 25)):
    """Optimized volume drop simulation"""
    db_drop = random.uniform(*db_range)
    scale = 10 ** (-db_drop / 20)
    
    # Simple linear fade
    fade = np.linspace(1, scale, X.shape[1])[None, :, None]
    X *= fade
    return np.clip(X, 0, 1)

def signal_loss_augment(X):
    """Simple signal loss simulation"""
    if random.random() < 0.6:  # 60% chance to apply
        # Create 1-2 silence regions
        num_regions = random.randint(1, 2)
        for _ in range(num_regions):
            start = random.randint(0, max(1, X.shape[1] - 15))
            width = random.randint(3, min(15, X.shape[1] - start))
            X[:, start:start + width, 0] = 0
    return X

def smart_mixup(X1, y1, X2, y2, alpha=0.2):
    """Smart mixup that preserves noise timing"""
    if random.random() < 0.3:  # 30% chance
        lam = np.random.beta(alpha, alpha)
        X_mixed = lam * X1 + (1 - lam) * X2
        # For labels, take max to preserve all noise events
        y_mixed = np.maximum(y1, y2 * (1 - lam))
        return X_mixed, y_mixed
    return X1, y1

# ───────── CPU-OPTIMIZED DATA GENERATOR ────────────────────────────────────
def gen(idxs, augment=True):
    """CPU-optimized generator with smart augmentation"""
    while True:
        np.random.shuffle(idxs)
        
        # For mixup, we need pairs of samples
        sample_cache = []
        
        for i in idxs:
            X, y = load_xy(i)
            
            # Skip empty or corrupted files
            if X.shape[1] <= 1 or y.shape[0] <= 1: 
                continue
                
            sample_cache.append((X, y))
            
            # Original sample
            yield X, y
            
            # Apply augmentation only if there's actual noise
            if augment and np.any(y > 0.5):
                # Create augmented version
                Xa, ya = X.copy(), y.copy()
                
                # Apply 2-3 random augmentations instead of all
                aug_list = []
                
                # Base augmentations (lighter for CPU)
                if random.random() < 0.8:
                    Xa = additive_noise(Xa)
                    aug_list.append("noise")
                
                if random.random() < 0.7:
                    Xa = random_intensity(Xa)
                    aug_list.append("intensity")
                
                if random.random() < 0.6:
                    Xa, ya = time_shift(Xa, ya, max_shift=10)
                    aug_list.append("shift")
                
                # Masking (lighter for CPU)
                if random.random() < 0.4:
                    Xa = frequency_mask(Xa, mask_prob=0.2)
                    aug_list.append("freq_mask")
                
                if random.random() < 0.3:
                    Xa = time_mask(Xa, mask_prob=0.15)
                    aug_list.append("time_mask")
                
                # Noise-type-specific augmentations (less frequent)
                if random.random() < 0.4:
                    if ya[:, 2].sum() > 0:  # compression_artifact
                        Xa = compression_augment(Xa)
                        aug_list.append("compression")
                    elif ya[:, 1].sum() > 0:  # volume_drop  
                        Xa = volume_drop_augment(Xa)
                        aug_list.append("volume")
                    elif ya[:, 0].sum() > 0:  # signal_loss
                        Xa = signal_loss_augment(Xa)
                        aug_list.append("signal_loss")
                
                # Smart mixup (occasionally)
                if len(sample_cache) > 1 and random.random() < 0.2:
                    X2, y2 = random.choice(sample_cache[:-1])
                    if X2.shape == Xa.shape and y2.shape == ya.shape:
                        Xa, ya = smart_mixup(Xa, ya, X2, y2)
                        aug_list.append("mixup")
                
                yield Xa, ya
                
                # Limit cache size for memory
                if len(sample_cache) > 10:
                    sample_cache = sample_cache[-5:]

# ───────── TENSORFLOW DATASET CREATION ───────────────────────────────
sig = (tf.TensorSpec((64, None, 1), tf.float32),
       tf.TensorSpec((None, 3), tf.float32))

print("🚀 Creating CPU-optimized training dataset...")
train_ds = (tf.data.Dataset.from_generator(lambda: gen(train_idx, True),
            output_signature=sig)
            .shuffle(512)  # Smaller shuffle buffer for CPU
            .padded_batch(BATCH_SIZE)
            .prefetch(2))  # Less prefetching for CPU

print("🔍 Creating validation dataset...")
val_ds = (tf.data.Dataset.from_generator(lambda: gen(val_idx, False),
          output_signature=sig)
          .padded_batch(BATCH_SIZE)
          .prefetch(2))

print("✅ CPU-optimized datasets ready!")
print("Improvements:")
print("  ✓ Reduced batch size (4 instead of 6)")
print("  ✓ Lighter augmentations for CPU")
print("  ✓ Smart mixup for better generalization")
print("  ✓ Selective augmentation (2-3 per sample)")
print("  ✓ Memory-efficient caching")

c:\Users\Yonatan\anaconda3\envs\tf-gpu\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


📊 Total available files: 2200
🎯 Training files: 1870 (85.0%)
📋 Validation files: 330 (15.0%)
🚀 Creating CPU-optimized training dataset...
🔍 Creating validation dataset...
✅ CPU-optimized datasets ready!
Improvements:
  ✓ Reduced batch size (4 instead of 6)
  ✓ Lighter augmentations for CPU
  ✓ Smart mixup for better generalization
  ✓ Selective augmentation (2-3 per sample)
  ✓ Memory-efficient caching


In [2]:
from tensorflow.keras import layers, models
import tensorflow as tf

# ═══════════════════════════════════════════════════════════════════
# 🧠 ADVANCED AUDIO NOISE DETECTION MODEL ARCHITECTURE
# ═══════════════════════════════════════════════════════════════════

def squeeze_excite_block(input_tensor, ratio=16, name_prefix="se"):
    """Enhanced Squeeze-and-Excitation block for better feature attention"""
    channel = input_tensor.shape[-1]
    
    # Global context
    se = layers.GlobalAveragePooling1D(name=f"{name_prefix}_gap")(input_tensor)
    se = layers.Dense(max(channel // ratio, 8), activation='relu', name=f"{name_prefix}_dense1")(se)
    se = layers.Dense(channel, activation='sigmoid', name=f"{name_prefix}_dense2")(se)
    se = layers.Reshape((1, channel), name=f"{name_prefix}_reshape")(se)
    
    # Apply attention weights
    attended = layers.Multiply(name=f"{name_prefix}_multiply")([input_tensor, se])
    return attended

def multi_head_attention_block(input_tensor, num_heads=4, key_dim=32, name_prefix="mha"):
    """Multi-head self-attention for temporal dependencies"""
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=key_dim,
        dropout=0.1,
        name=f"{name_prefix}_attention"
    )(input_tensor, input_tensor)
    
    # Add & Norm
    attention_output = layers.Add(name=f"{name_prefix}_add")([input_tensor, attention_output])
    attention_output = layers.LayerNormalization(name=f"{name_prefix}_norm")(attention_output)
    
    return attention_output

def residual_conv_block(x, filters, kernel_size=3, dilation_rate=1, block_name="res"):
    """Residual convolutional block with dilated convolutions"""
    shortcut = x
    
    # First conv
    x = layers.Conv2D(filters, kernel_size, padding='same', 
                      dilation_rate=dilation_rate, activation='relu',
                      name=f"{block_name}_conv1")(x)
    x = layers.BatchNormalization(name=f"{block_name}_bn1")(x)
    x = layers.Dropout(0.1, name=f"{block_name}_dropout1")(x)
    
    # Second conv  
    x = layers.Conv2D(filters, kernel_size, padding='same',
                      dilation_rate=dilation_rate, activation='relu',
                      name=f"{block_name}_conv2")(x)
    x = layers.BatchNormalization(name=f"{block_name}_bn2")(x)
    
    # Adjust shortcut if needed
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, padding='same', name=f"{block_name}_shortcut_conv")(shortcut)
        shortcut = layers.BatchNormalization(name=f"{block_name}_shortcut_bn")(shortcut)
    
    # Residual connection
    x = layers.Add(name=f"{block_name}_add")([x, shortcut])
    x = layers.Activation('relu', name=f"{block_name}_activation")(x)
    
    return x

def build_advanced_model(shape=(64, None, 1)):
    """
    🎯 Advanced Multi-Scale Audio Noise Detection Model
    
    Features:
    - Multi-scale CNN feature extraction
    - Residual connections for deep networks  
    - Attention mechanisms for temporal modeling
    - Squeeze-and-Excitation for feature selection
    - Bidirectional LSTM for sequence modeling
    - Multi-head attention for long-range dependencies
    """
    
    inputs = layers.Input(shape, name='spectrogram_input')
    
    # ───────── MULTI-SCALE FEATURE EXTRACTION ───────────────────────
    
    # Scale 1: Fine-grained features (small receptive field)
    x1 = layers.Conv2D(32, (3, 3), padding='same', activation='relu', name='conv1_fine')(inputs)
    x1 = layers.BatchNormalization(name='bn1_fine')(x1)
    x1 = residual_conv_block(x1, 32, kernel_size=3, dilation_rate=1, block_name="res1")
    x1 = layers.MaxPooling2D((2, 1), name='pool1')(x1)  # Reduce frequency, keep time
    
    # Scale 2: Medium-grained features  
    x2 = layers.Conv2D(48, (5, 3), padding='same', activation='relu', name='conv2_medium')(x1)
    x2 = layers.BatchNormalization(name='bn2_medium')(x2)
    x2 = residual_conv_block(x2, 48, kernel_size=3, dilation_rate=2, block_name="res2")
    x2 = layers.MaxPooling2D((2, 1), name='pool2')(x2)
    
    # Scale 3: Coarse-grained features (large receptive field)
    x3 = layers.Conv2D(64, (7, 3), padding='same', activation='relu', name='conv3_coarse')(x2)  
    x3 = layers.BatchNormalization(name='bn3_coarse')(x3)
    x3 = residual_conv_block(x3, 64, kernel_size=3, dilation_rate=4, block_name="res3")
    x3 = layers.MaxPooling2D((2, 1), name='pool3')(x3)
    
    # Additional deep layers
    x4 = residual_conv_block(x3, 80, kernel_size=3, dilation_rate=1, block_name="res4")
    x4 = layers.Dropout(0.15, name='dropout4')(x4)
    
    x5 = residual_conv_block(x4, 96, kernel_size=3, dilation_rate=2, block_name="res5")
    x5 = layers.Dropout(0.15, name='dropout5')(x5)
    
    # ───────── TEMPORAL SEQUENCE MODELING ────────────────────────────
    
    # Reshape for sequence modeling: (batch, time, freq*channels)
    x = layers.Permute((2, 1, 3), name='permute_for_temporal')(x5)  # (batch, time, freq, channels)
    time_steps = tf.shape(x)[1]
    freq_channels = x.shape[2] * x.shape[3] 
    x = layers.Reshape((-1, freq_channels), name='reshape_for_temporal')(x)
    
    # Dense projection for dimensionality control
    x = layers.Dense(128, activation='relu', name='temporal_projection')(x)
    x = layers.Dropout(0.2, name='temporal_dropout')(x)
    
    # ───────── ATTENTION-BASED TEMPORAL PROCESSING ──────────────────
    
    # Multi-head self-attention for long-range dependencies
    x = multi_head_attention_block(x, num_heads=8, key_dim=16, name_prefix="mha1")
    
    # Squeeze-and-Excitation for feature attention
    x = squeeze_excite_block(x, ratio=8, name_prefix="se1")
    
    # Bidirectional LSTM for sequence modeling
    x = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True, recurrent_dropout=0.1),
        name='bidirectional_lstm_1'
    )(x)
    x = layers.LayerNormalization(name='layer_norm1')(x)
    x = layers.Dropout(0.2, name='lstm_dropout1')(x)
    
    # Second LSTM layer for deeper temporal understanding
    x = layers.Bidirectional(
        layers.LSTM(48, return_sequences=True, recurrent_dropout=0.1),
        name='bidirectional_lstm_2'  
    )(x)
    x = layers.LayerNormalization(name='layer_norm2')(x)
    
    # Final attention layer
    x = multi_head_attention_block(x, num_heads=4, key_dim=24, name_prefix="mha2")
    
    # ───────── OUTPUT LAYERS ──────────────────────────────────────────
    
    # Dense layers for final classification
    x = layers.Dense(64, activation='relu', name='final_dense_1')(x)
    x = layers.Dropout(0.25, name='final_dropout_1')(x)
    
    x = layers.Dense(32, activation='relu', name='final_dense_2')(x)
    x = layers.Dropout(0.15, name='final_dropout_2')(x)
    
    # Multi-label output (3 noise types)
    outputs = layers.Dense(3, activation='sigmoid', name='noise_classification')(x)
    
    # Create and return model
    model = models.Model(inputs, outputs, name='AdvancedNoiseDetector')
    
    return model

# ───────── MODEL CREATION ─────────────────────────────────────────────

print("🏗️ Building advanced noise detection model...")
model = build_advanced_model(shape=(64, None, 1))

print(f"📊 Model Parameters: {model.count_params():,}")
print("🎯 Model Architecture:")
model.summary()

print("\n✅ Advanced model ready for training!")

🏗️ Building advanced noise detection model...
📊 Model Parameters: 876,851
🎯 Model Architecture:
Model: "AdvancedNoiseDetector"
__________________________________________________________________________________________________
📊 Model Parameters: 876,851
🎯 Model Architecture:
Model: "AdvancedNoiseDetector"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 spectrogram_input (InputLayer)  [(None, 64, None, 1  0          []                               
                                )]                                                                
                                                                                                  
 conv1_fine (Conv2D)            (None, 64, None, 32  320         ['spectrogram_input[0][0]']      
                                )                                                                 


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os

# Disable mixed precision for CPU
tf.keras.mixed_precision.set_global_policy('float32')

def focal_loss_dynamic(gamma=2.0, eps=1e-7):
    """
    Advanced Focal Loss with Dynamic Class Weighting
    Designed for severe class imbalance between noise (5%) and clean (95%)
    """
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1-eps)
        
        # Calculate dynamic class frequency
        freq_pos = tf.reduce_mean(y_true, axis=[0,1])
        alpha = 1 - freq_pos  # Higher weight for minority class
        
        # Focal Loss with dynamic alpha
        term1 = - alpha * y_true * tf.pow(1-y_pred, gamma) * tf.math.log(y_pred)
        term2 = - (1-alpha) * (1-y_true) * tf.pow(y_pred, gamma) * tf.math.log(1-y_pred)
        return tf.reduce_mean(term1+term2)
    return loss_fn

def f1_metric_05(y_true, y_pred):
    """Simple F1 Score with 0.5 threshold"""
    y_pred = tf.cast(y_pred>0.5, tf.float32)
    tp = tf.reduce_sum(y_true*y_pred)
    fp = tf.reduce_sum((1-y_true)*y_pred)
    fn = tf.reduce_sum(y_true*(1-y_pred))
    prec = tp/(tp+fp+1e-7)
    rec = tp/(tp+fn+1e-7)
    return 2*prec*rec/(prec+rec+1e-7)

# Set function name for metric
f1_metric_05.__name__ = "f1_metric_05"

print("Setting up improved loss function and metrics...")

# Simple optimizer - stable for training
optimizer = tf.keras.optimizers.Adam(
    learning_rate=8e-5,  # Lower learning rate for stability
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# Compile model with optimized settings
model.compile(
    optimizer=optimizer,
    loss=focal_loss_dynamic(gamma=2.0),  # Balanced gamma
    metrics=[
        f1_metric_05,
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
)

# Training configuration
total_training_files = len(train_idx)
total_validation_files = len(val_idx)

# Optimized steps for training
steps_per_epoch = min(400, (total_training_files * 2) // BATCH_SIZE)
validation_steps = min(50, total_validation_files // BATCH_SIZE)

print(f"Training configuration:")
print(f"   Steps per epoch: {steps_per_epoch}")
print(f"   Validation steps: {validation_steps}")
print(f"   Training files: {total_training_files}")
print(f"   Validation files: {total_validation_files}")
print(f"   Batch size: {BATCH_SIZE}")

# Advanced callbacks with proper model saving
callbacks = [
    # Save best model - .keras format (primary)
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "best_model.keras"),
        save_best_only=True, 
        monitor='val_f1_metric_05', 
        mode='max', 
        verbose=1,
        save_format='keras'
    ),
    
    # Save best model - .h5 format (compatibility)
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "best_model.h5"),
        save_best_only=True,
        monitor='val_f1_metric_05',
        mode='max', 
        verbose=1,
        save_format='h5'
    ),
    
    # Early stopping with patience
    tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_metric_05',
        mode='max',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Learning rate reduction
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_f1_metric_05',
        mode='max',
        factor=0.7,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

print("Model compilation completed!")
print("Ready to start training...")

# Model training
print(f"\nStarting training with {model.count_params():,} parameters...")

history = model.fit(
    train_ds,
    epochs=20,  # Adequate epochs for convergence
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

# Save final model in BOTH formats with confirmation
print("\nSaving final model in both formats...")

# Save as .keras (TensorFlow recommended format)
keras_path = os.path.join(MODEL_DIR, "final_model.keras")
model.save(keras_path, save_format='keras')
print(f"✅ Model saved as Keras format: {keras_path}")

# Save as .h5 (Compatibility format)
h5_path = os.path.join(MODEL_DIR, "final_model.h5")
model.save(h5_path, save_format='h5')
print(f"✅ Model saved as H5 format: {h5_path}")

# Verify both files exist
if os.path.exists(keras_path) and os.path.exists(h5_path):
    keras_size = os.path.getsize(keras_path) / (1024*1024)  # MB
    h5_size = os.path.getsize(h5_path) / (1024*1024)  # MB
    print(f"📁 Keras file size: {keras_size:.1f} MB")
    print(f"📁 H5 file size: {h5_size:.1f} MB")
    print("🎉 Both model formats saved successfully!")
else:
    print("⚠️ Warning: One or both model files may not have saved correctly")

# Results visualization
plt.figure(figsize=(12, 8))

# F1 Score
plt.subplot(2, 2, 1)
if 'f1_metric_05' in history.history:
    plt.plot(history.history['f1_metric_05'], label='Train F1', linewidth=2)
if 'val_f1_metric_05' in history.history:
    plt.plot(history.history['val_f1_metric_05'], label='Val F1', linewidth=2)
plt.title('F1 Score Progress', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss
plt.subplot(2, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
plt.title('Training & Validation Loss', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Precision
plt.subplot(2, 2, 3)
if 'precision' in history.history:
    plt.plot(history.history['precision'], label='Train Precision', linewidth=2)
if 'val_precision' in history.history:
    plt.plot(history.history['val_precision'], label='Val Precision', linewidth=2)
plt.title('Precision', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True, alpha=0.3)

# Recall
plt.subplot(2, 2, 4)
if 'recall' in history.history:
    plt.plot(history.history['recall'], label='Train Recall', linewidth=2)
if 'val_recall' in history.history:
    plt.plot(history.history['val_recall'], label='Val Recall', linewidth=2)
plt.title('Recall', fontsize=12, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Results summary
print("\nTraining completed successfully!")
if 'val_f1_metric_05' in history.history:
    best_f1 = max(history.history['val_f1_metric_05'])
    print(f"Best validation F1 Score: {best_f1:.4f}")

if 'val_accuracy' in history.history:
    best_acc = max(history.history['val_accuracy'])
    print(f"Best validation accuracy: {best_acc:.4f}")

print(f"\nModels saved in directory: {MODEL_DIR}")
print("Available model files:")
for file in os.listdir(MODEL_DIR):
    if file.endswith(('.keras', '.h5')):
        file_path = os.path.join(MODEL_DIR, file)
        size_mb = os.path.getsize(file_path) / (1024*1024)
        print(f"   📄 {file} ({size_mb:.1f} MB)")

print("\n🚀 Model ready for noise detection in new audio files!")

Setting up improved loss function and metrics...
Training configuration:
   Steps per epoch: 400
   Validation steps: 50
   Training files: 1870
   Validation files: 330
   Batch size: 4
Model compilation completed!
Ready to start training...

Starting training with 876,851 parameters...
Epoch 1/20
400/400 [==============================] - ETA: 0s - loss: 0.0036 - f1_metric_05: 0.2763 - precision: 0.2034 - recall: 0.9110 - accuracy: 0.8933
Epoch 1: val_f1_metric_05 improved from -inf to 0.19269, saving model to models\best_model.keras

Epoch 1: val_f1_metric_05 improved from -inf to 0.19269, saving model to models\best_model.keras

Epoch 1: val_f1_metric_05 improved from -inf to 0.19269, saving model to models\best_model.h5

Epoch 1: val_f1_metric_05 improved from -inf to 0.19269, saving model to models\best_model.h5
400/400 [==============================] - 1484s 3s/step - loss: 0.0036 - f1_metric_05: 0.2763 - precision: 0.2034 - recall: 0.9110 - accuracy: 0.8933 - val_loss: 0.0057 

: 

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# 🔄 LOAD TRAINED MODEL
# ═══════════════════════════════════════════════════════════════════

import tensorflow as tf
import os
import numpy as np

print("Loading trained model...")

# Define custom objects
def focal_loss_dynamic(gamma=2.0, eps=1e-7):
    """Advanced Focal Loss with Dynamic Class Weighting"""
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1-eps)
        freq_pos = tf.reduce_mean(y_true, axis=[0,1])
        alpha = 1 - freq_pos
        term1 = - alpha * y_true * tf.pow(1-y_pred, gamma) * tf.math.log(y_pred)
        term2 = - (1-alpha) * (1-y_true) * tf.pow(y_pred, gamma) * tf.math.log(1-y_pred)
        return tf.reduce_mean(term1+term2)
    return loss_fn

def f1_metric_05(y_true, y_pred):
    """F1 Score with 0.5 threshold"""
    y_pred = tf.cast(y_pred>0.5, tf.float32)
    tp = tf.reduce_sum(y_true*y_pred)
    fp = tf.reduce_sum((1-y_true)*y_pred)
    fn = tf.reduce_sum(y_true*(1-y_pred))
    prec = tp/(tp+fp+1e-7)
    rec = tp/(tp+fn+1e-7)
    return 2*prec*rec/(prec+rec+1e-7)

f1_metric_05.__name__ = "f1_metric_05"

# Load model
MODEL_DIR = "models"
model_path = os.path.join(MODEL_DIR, "final_model.h5")

custom_objects = {
    'focal_loss_dynamic': focal_loss_dynamic,
    'loss_fn': focal_loss_dynamic(),
    'f1_metric_05': f1_metric_05
}

try:
    model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
    print(f"✓ Model loaded successfully ({model.count_params():,} parameters)")
    
    # Quick test
    test_input = np.random.random((1, 64, 100, 1)).astype(np.float32)
    test_output = model.predict(test_input, verbose=0)
    print(f"✓ Model ready for inference")
    
except Exception as e:
    print(f"✗ Error loading model: {e}")
    model = None

c:\Users\Yonatan\anaconda3\envs\tf-gpu\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


🔄 הגדרת פונקציות מותאמות אישית...
🔍 בדיקת קבצי מודל זמינים...
קבצי מודלים זמינים: ['best_model.h5', 'best_model.keras', 'final_model.h5', 'final_model.keras']
   📄 best_model.h5 (10.5 MB) - 15/08/2025 23:21
   📄 best_model.keras (10.5 MB) - 15/08/2025 23:21
   📄 final_model.h5 (10.5 MB) - 16/08/2025 00:45
   📄 final_model.keras (10.5 MB) - 16/08/2025 00:45

🔄 מנסה לטעון מודל מ: models\final_model.h5
✅ המודל נטען בהצלחה!
📊 פרמטרים במודל: 876,851
🔧 Input shape: (None, 64, None, 1)
🔧 Output shape: (None, None, 3)
✅ בדיקת חיזוי הצליחה - צורת פלט: (1, 100, 3)
📊 טווח ערכי פלט: [0.090, 0.210]

🎉 המודל נטען בהצלחה מ: models\final_model.h5
🎯 המודל מוכן לזיהוי רעשים!
המשתנה 'model' מכיל את המודל הטעון

📋 פרטי המודל:
   שם המודל: AdvancedNoiseDetector
   מספר שכבות: 76
   פרמטרים: 876,851


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# 📊 MODEL VALIDATION TEST
# ═══════════════════════════════════════════════════════════════════

import numpy as np
import os

print("Validating model on test data...")

# Setup validation data (same as training split)
DATA_DIR = "preprocessed"
X_files = sorted(f for f in os.listdir(DATA_DIR) if f.endswith("_X.npy"))
total_files = len(X_files)
val_split = int(0.85 * total_files)

np.random.seed(42)
idx = np.arange(total_files)
np.random.shuffle(idx)
val_idx = idx[val_split:]

def load_xy(i):
    X = np.load(os.path.join(DATA_DIR, X_files[i])).astype(np.float32)
    y = np.load(os.path.join(DATA_DIR, f"{X_files[i][:-6]}_y.npy")).astype(np.float32)
    
    if y.shape[0] == 3 and y.shape[1] != 3: 
        y = y.T
    
    MAX_T = 800
    actual_T = min(X.shape[1], y.shape[0], MAX_T)
    X = X[:, :actual_T]
    y = y[:actual_T]
    
    return X[..., None], y

# Test on 20 validation files
num_test_files = min(20, len(val_idx))
test_indices = val_idx[:num_test_files]

all_predictions = []
all_true_labels = []

for file_idx in test_indices:
    try:
        X, y_true = load_xy(file_idx)
        X_input = X[np.newaxis, :]
        y_pred = model.predict(X_input, verbose=0)[0]
        
        min_length = min(y_true.shape[0], y_pred.shape[0])
        y_true = y_true[:min_length]
        y_pred = y_pred[:min_length]
        
        all_predictions.append(y_pred)
        all_true_labels.append(y_true)
        
    except Exception as e:
        continue

# Analyze results
if all_predictions:
    all_pred_concat = np.vstack(all_predictions)
    all_true_concat = np.vstack(all_true_labels)
    
    THRESHOLD = 0.8
    predictions_binary = (all_pred_concat > THRESHOLD).astype(int)
    true_labels_binary = (all_true_concat > 0.5).astype(int)
    
    total_frames = len(all_true_concat)
    frames_clean = np.sum(~np.any(true_labels_binary, axis=1))
    frames_with_noise = np.sum(np.any(true_labels_binary, axis=1))
    
    false_positives = np.sum(np.any(predictions_binary, axis=1) & ~np.any(true_labels_binary, axis=1))
    false_negatives = np.sum(np.any(true_labels_binary, axis=1) & ~np.any(predictions_binary, axis=1))
    
    fp_rate = false_positives / frames_clean * 100
    
    print(f"Total frames: {total_frames:,}")
    print(f"Clean frames: {frames_clean:,}, Noise frames: {frames_with_noise:,}")
    print(f"False Positives: {false_positives:,} ({fp_rate:.3f}%)")
    print(f"False Negatives: {false_negatives:,}")
    
    # Per-noise type performance
    noise_names = ['signal_loss', 'volume_drop', 'compression_artifact']
    total_detected = 0
    total_true = 0
    
    for i, noise_name in enumerate(noise_names):
        true_noise = true_labels_binary[:, i]
        pred_noise = predictions_binary[:, i]
        
        tp = np.sum((true_noise == 1) & (pred_noise == 1))
        true_count = np.sum(true_noise)
        
        total_detected += tp
        total_true += true_count
        
        recall = tp / (true_count + 1e-7)
        print(f"{noise_name}: {tp}/{true_count} detected (recall: {recall:.3f})")
    
    overall_recall = total_detected / total_true if total_true > 0 else 0
    print(f"Overall recall: {overall_recall:.3f}, FP rate: {fp_rate:.3f}%")
    print("✓ Model validation complete")

else:
    print("✗ No files processed successfully")

🎯 הערכת מודל מלאה - כל הנתונים מהתחלה
📊 נתוני בסיס:
   סה"כ פריימים שנבדקו: 16,000
   פריימים נקיים (ללא רעש): 14,554 (91.0%)
   פריימים עם רעש: 1,446 (9.0%)

🔍 תוצאות הזיהוי (סף 0.8):
   פריימים שזוהו כנקיים: 14,548
   פריימים שזוהו כרועשים: 1,452

⚖️  מטריקס בלבול כללי (רועש/נקי):
   True Positive (רעש זוהה נכון): 1,446
   False Positive (נקי שזוהה כרועש): 6
   True Negative (נקי זוהה נכון): 14,548
   False Negative (רעש שלא זוהה): 0

📈 שיעורי ביצועים:
   False Positive Rate: 0.041% (6 מתוך 14,554)
   False Negative Rate: 0.000% (0 מתוך 1,446)
   דיוק כללי: 99.96%

🎯 פירוט לכל סוג רעש:
   signal_loss:
      רעשים אמיתיים: 427 פריימים
      רעשים שזוהו: 427 פריימים
      זוהו נכון (TP): 427
      זוהו בטעות (FP): 0
      לא זוהו (FN): 0
      Precision: 1.000, Recall: 1.000
   volume_drop:
      רעשים אמיתיים: 323 פריימים
      רעשים שזוהו: 323 פריימים
      זוהו נכון (TP): 323
      זוהו בטעות (FP): 0
      לא זוהו (FN): 0
      Precision: 1.000, Recall: 1.000
   compression_artifact